In [1]:

import pandas as pd # type: ignore
import numpy as np # type: ignore

In [2]:
inkar = pd.read_csv("../data/inkar_2022.csv",sep=";",decimal=",",dtype={'Kennziffer':str,'Kennziffer_EU':str,'Name':str}, low_memory=False)
inkar.head()


,Bereich,ID,Indikator,Raumbezug,Kennziffer,Name,Zeitbezug,Wert
0,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,001,Flensburg,1995,10494.0
1,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,001,Flensburg,1996,11921.0
2,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,001,Flensburg,1997,13213.0
3,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,001,Flensburg,1998,13301.0
4,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,001,Flensburg,1999,12048.0


In [3]:
indicators = inkar.Indikator.unique()
categorys = inkar.Bereich.unique()
ids = inkar.ID.unique()
#print(indicators)
#print(categorys)
#print(ids)
zeit = inkar.Zeitbezug.unique()
raumbezug = inkar.Raumbezug.unique()

print(raumbezug)

['Arbeitsmarktregionen' 'BBSR-Mittelbereiche' 'Braunkohlereviere' 'Bund'
 'Bundesländer' 'Europäische Metropolregion' 'Gemeindeverbände'
 'Großstadtregionaler Einzugsbereich' 'Großstadtregionen' 'IHK-Bezirke'
 'Kreise' 'Kreisregionen' 'Raumordnungsregionen'
 'Raumtyp Bezug Lage (Kreise)' 'Raumtyp Bezug: Lage (Gemeindeverbände)'
 'Regierungsbezirke' 'Regionalstatistischer Raumtyp 17 (RegioStaR17)'
 'Regionalstatistischer Raumtyp 2 (RegioStaR2)'
 'Regionalstatistischer Raumtyp 7 (RegioStaR7)'
 'Regionalstatistischer Raumtyp Gemeindetyp 5 (Regio'
 'Siedlungsstruktureller Kreistyp'
 'Siedlungsstruktureller Kreistyp (Bundesländer)'
 'Siedlungsstruktureller Kreistyp (West-Ost)'
 'Siedlungsstruktureller Regionstyp'
 'Siedlungsstruktureller Regionstyp (Bundesländer)'
 'Siedlungsstruktureller Regionstyp (West-Ost)' 'Stadt- und Gemeindetyp'
 'Stadt- und Gemeindetyp (Bundesländer)'
 'Stadt- und Gemeindetyp (West-Ost)'
 'Stadt- und Gemeindetyp differenziert' 'Städtischer-Ländlicher Raum'
 'Städtis

In [ ]:

# Wähle nur Daten aus, die entweder "Kreise" oder "Gemeinden" als Raumbezug haben
raumbezug_selection = ["Kreise", "Gemeinden"]
temp = inkar[inkar['Raumbezug'].isin(raumbezug_selection)]

zeitbezug_min_max = temp.groupby(['Bereich', 'ID', 'Indikator', 'Raumbezug'])['Zeitbezug'].agg([('min', 'min'), ('max', 'max')])

zeitbezug = pd.pivot_table(zeitbezug_min_max, index=['Bereich', 'ID', 'Indikator'], columns='Raumbezug', values=['min', 'max']).reset_index()

zeitbezug.columns = ['Bereich', 'ID', 'Indikator', 'Min_Gemeinden', 'Min_Kreise', 'Max_Gemeinden', 'Max_Kreise']

zeitbezug['Zeitbezug_Gemeinden'] = zeitbezug.apply(lambda row: f"{int(row['Max_Gemeinden'])} - {int(row['Min_Gemeinden'])}" if not pd.isna(row['Min_Gemeinden']) and not pd.isna(row['Max_Gemeinden']) else '', axis=1)
zeitbezug['Zeitbezug_Kreise'] = zeitbezug.apply(lambda row: f"{int(row['Max_Kreise'])} - {int(row['Min_Kreise'])}" if not pd.isna(row['Min_Kreise']) and not pd.isna(row['Max_Kreise']) else '', axis=1)

zeitbezug['Gemeinden'] = zeitbezug['Max_Gemeinden'].apply(lambda x: 'X' if not pd.isna(x) else '')
zeitbezug['Kreise'] = zeitbezug['Max_Kreise'].apply(lambda x: 'X' if not pd.isna(x) else '')

data = zeitbezug[['Bereich', 'ID', 'Indikator', 'Gemeinden', 'Kreise', 'Zeitbezug_Gemeinden', 'Zeitbezug_Kreise']]

data


In [ ]:


data = data.assign(
    select="",
    category="",
    category_de=data['Bereich'],
    category_nl="",
    name="",
    name_de=data['Indikator'],
    algorithmus="",
    description="",
    statistical_basis="",
    name_nl="",
    source_de="Inkar",
    source_id_de=data['ID'],
    source_nl="",
    source_id_nl="",
    gemeinden=data['Gemeinden'],
    kreise=data['Kreise'],
    zeitbezug_gemeinden=data['Zeitbezug_Gemeinden'],
    zeitbezug_kreise=data['Zeitbezug_Kreise'],
)

column_order = ['select', 'category', 'category_de', 'category_nl', 'name', 'name_de', "algorithmus", "description", "statistical_basis", 'name_nl', 'source_de', 'source_id_de', 'source_nl', 'source_id_nl', 'gemeinden', 'kreise', 'zeitbezug_gemeinden', 'zeitbezug_kreise']
data = data[column_order]

data.reset_index(inplace=True, drop=True)

data.to_csv("C:/Users/juliu/VSCODE/VSCODE/inkar-indicators/src/out/new.csv")

In [ ]:
# OLD OLD OLD OLD OLD OLD OLD

raumbezug_selection = ["Kreise","Gemeinden", "Gemeindeverbände"]
temp = inkar[inkar['Raumbezug'].isin(raumbezug_selection)]

menu = temp.groupby(['Bereich','Indikator','ID']).aggregate({'Zeitbezug':['min','max']})
menu.columns = ["_".join(a) for a in menu.columns.to_flat_index()]
menu = menu.merge(temp.pivot_table(index=['Bereich','Indikator','ID'], columns='Raumbezug',values='Kennziffer',aggfunc=np.count_nonzero),left_index=True,right_index=True)

for column in raumbezug_selection:
    menu[column] = (menu[column]>0).replace({True: 'X', False: ""})

print(menu.index.get_level_values('Bereich'))

menu = menu.assign(
    select="",
    category="",
    category_de=menu.index.get_level_values('Bereich'),
    category_nl="",
    name="",
    name_de=menu.index.get_level_values('Indikator'),
    algorithmus="",
    description="",
    statistical_basis="",
    name_nl="",
    source_de="Inkar",
    source_id_de=menu.index.get_level_values('ID'),
    source_nl="",
    source_id_nl="",
)

column_order = ['select', 'category', 'category_de', 'category_nl', 'name', 'name_de', "algorithmus", "description", "statistical_basis", 'name_nl', 'source_de', 'source_id_de', 'source_nl', 'source_id_nl', 'Zeitbezug_min', 'Zeitbezug_max', 'Gemeinden', 'Gemeindeverbände', 'Kreise']
menu = menu[column_order]

menu.reset_index(inplace=True, drop=True)

#menu.to_csv("C:/Users/juliu/VSCODE/VSCODE/inkar-indicators/src/out/menu.csv")
#menu.to_excel("C:/Users/juliu/VSCODE/VSCODE/inkar-indicators/src/out/output1.xlsx", index=False)
menu

